In [6]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import folium
from typing import List, Tuple, Dict, Optional
import random

class ParkingOptimizer:
    def __init__(self, parking_data: pd.DataFrame, destination: Tuple[float, float],
                 weights: Dict[str, float] = None):
        """
        Initialize the parking optimizer

        Args:
            parking_data: DataFrame with columns [id, lat, lng, rate_hourly]
            destination: Tuple of (latitude, longitude) for destination
            weights: Dictionary of weights for different factors (distance, cost, accessibility)
        """
        # Store initial parameters
        self.parking_data = parking_data
        self.destination = destination
        self.weights = weights or {'distance': 0.5, 'cost': 0.3, 'accessibility': 0.2}

        # Calculate distances for all parking lots
        self.parking_data['distance'] = self.parking_data.apply(
            lambda row: self.haversine_distance(
                row['lat'], row['lng'],
                self.destination[0], self.destination[1]
            ),
            axis=1
        )

        # PSO algorithm parameters
        self.num_particles = 20
        self.max_iterations = 50
        self.c1 = 2.0  # cognitive parameter
        self.c2 = 2.0  # social parameter
        self.w = 0.7   # inertia weight

    def haversine_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
        """
        Calculate the distance between two points on Earth using Haversine formula

        Args:
            lat1, lon1: Coordinates of first point
            lat2, lon2: Coordinates of second point

        Returns:
            float: Distance in meters between the two points
        """
        R = 6371000  # Earth's radius in meters

        # Convert decimal degrees to radians
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Haversine formula components
        dlat = lat2 - lat1
        dlon = lon2 - lon1

        # Calculate distance
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        distance = R * c

        return distance

    def _filter_by_distance(self, radius: float) -> pd.DataFrame:
        """
        Filter parking lots within specified radius of destination

        Args:
            radius: Search radius in meters

        Returns:
            DataFrame containing only parking lots within the radius
        """
        return self.parking_data[self.parking_data['distance'] <= radius].copy()

    def calculate_fitness(self, parking_lot: pd.Series) -> float:
        """
        Calculate fitness score for a parking lot

        Args:
            parking_lot: Series containing parking lot data

        Returns:
            float: Fitness score between 0 and 1
        """
        # Get maximum values for normalization
        max_distance = self.filtered_data['distance'].max()
        max_cost = self.filtered_data['rate_hourly'].max()

        # Calculate normalized distance score (closer = better)
        distance_score = (max_distance - parking_lot['distance']) / max_distance if max_distance > 0 else 1

        # Calculate normalized cost score (cheaper = better)
        cost_score = (max_cost - parking_lot['rate_hourly']) / max_cost if max_cost > 0 else 1

        # Calculate simplified accessibility score (would be more complex in real application)
        accessibility_score = random.uniform(0.7, 1.0)

        # Calculate weighted sum of all factors
        fitness = (
            self.weights['distance'] * distance_score +
            self.weights['cost'] * cost_score +
            self.weights['accessibility'] * accessibility_score
        )

        return fitness

    def initialize_particles(self) -> Tuple[List[int], List[float], List[float]]:
        """
        Initialize PSO particles

        Returns:
            Tuple containing:
            - List of particle positions (parking lot indices)
            - List of particle best fitness values
            - List of particle velocities
        """
        # Adjust number of particles if we have fewer parking lots than default
        self.num_particles = min(self.num_particles, len(self.filtered_data))

        # Initialize random positions, velocities, and fitness values
        positions = random.sample(range(len(self.filtered_data)), self.num_particles)
        velocities = [random.uniform(-1, 1) for _ in range(self.num_particles)]
        particle_best_fitness = [self.calculate_fitness(self.filtered_data.iloc[pos])
                               for pos in positions]

        return positions, particle_best_fitness, velocities

    def optimize(self) -> Optional[Tuple[pd.Series, int]]:
        """
        Run PSO optimization to find best parking lot

        Returns:
            Tuple containing best parking lot data and search radius if found,
            None if no parking lots are available
        """
        # Try 800m radius first
        self.filtered_data = self._filter_by_distance(800)
        if len(self.filtered_data) == 0:
            print("No parking lots are available within an 800-meter radius. Try expanding the search radius to 1600 meters...")
            # Try 1600m radius
            self.filtered_data = self._filter_by_distance(1600)
            if len(self.filtered_data) == 0:
                print("No parking lots are available within a 1600-meter radius.")
                return None
            search_radius = 1600
        else:
            search_radius = 800

        # Initialize PSO
        positions, particle_best_fitness, velocities = self.initialize_particles()
        particle_best_positions = positions.copy()

        # Initialize global best
        global_best_idx = np.argmax(particle_best_fitness)
        global_best_position = positions[global_best_idx]
        global_best_fitness = particle_best_fitness[global_best_idx]

        # Main optimization loop
        for _ in range(self.max_iterations):
            for i in range(self.num_particles):
                # Update velocity
                r1, r2 = random.random(), random.random()
                cognitive = self.c1 * r1 * (particle_best_positions[i] - positions[i])
                social = self.c2 * r2 * (global_best_position - positions[i])
                velocities[i] = self.w * velocities[i] + cognitive + social

                # Update position
                new_position = int(positions[i] + velocities[i]) % len(self.filtered_data)
                positions[i] = new_position

                # Calculate new fitness
                fitness = self.calculate_fitness(self.filtered_data.iloc[positions[i]])

                # Update particle best
                if fitness > particle_best_fitness[i]:
                    particle_best_fitness[i] = fitness
                    particle_best_positions[i] = positions[i]

                    # Update global best
                    if fitness > global_best_fitness:
                        global_best_fitness = fitness
                        global_best_position = positions[i]

        return self.filtered_data.iloc[global_best_position], search_radius

    def visualize_results(self, best_parking_lot: Optional[pd.Series],
                         search_radius: int) -> folium.Map:
        """
        Create a map visualization of parking lots and recommendation

        Args:
            best_parking_lot: Series containing best parking lot data (or None if none found)
            search_radius: The search radius used (800 or 1600 meters)

        Returns:
            folium.Map object
        """
        # Create map centered on destination
        m = folium.Map(location=[self.destination[0], self.destination[1]],
                      zoom_start=15)

        # Add destination marker
        folium.Marker(
            [self.destination[0], self.destination[1]],
            popup='Destination',
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(m)

        # Add circle showing search radius
        folium.Circle(
            radius=search_radius,
            location=[self.destination[0], self.destination[1]],
            color='red',
            fill=True,
            fillOpacity=0.1
        ).add_to(m)

        # First add all parking lots in black
        for _, lot in self.parking_data.iterrows():
            if best_parking_lot is not None and lot['id'] == best_parking_lot['id']:
                continue
            if lot['id'] in self.filtered_data['id'].values:
                continue

            popup_text = f"""
                ID: {lot['id']}<br>
                Rate: ${lot['rate_hourly']}/hr<br>
                Distance: {lot['distance']:.0f}m
            """
            folium.Marker(
                [lot['lat'], lot['lng']],
                popup=popup_text,
                icon=folium.Icon(color='black')
            ).add_to(m)

        # Add markers for parking lots within search radius
        if best_parking_lot is not None:
            for _, lot in self.filtered_data.iterrows():
                if lot['id'] == best_parking_lot['id']:
                    color = 'green'  # Best parking lot
                else:
                    color = 'blue'   # Candidate parking lots

                popup_text = f"""
                    ID: {lot['id']}<br>
                    Rate: ${lot['rate_hourly']}/hr<br>
                    Distance: {lot['distance']:.0f}m
                """
                folium.Marker(
                    [lot['lat'], lot['lng']],
                    popup=popup_text,
                    icon=folium.Icon(color=color)
                ).add_to(m)

        return m

In [17]:
# Load the data
parking_data = pd.read_csv('Parking_Lot_Dataset.csv')

# Set the destination coordinates (e.g., a location near downtown Toronto)
destination = (43.6532, -79.3832)

# Set the scoring weights
weights = {
    'distance': 0.5,    # 50% weight for distance
    'cost': 0.3,        # 30% weight for cost
    'accessibility': 0.2 # 20% weight for accessibility
}

# Create an instance of the optimizer
optimizer = ParkingOptimizer(parking_data, destination, weights)

# Run the optimization algorithm to find the best parking lot
result = optimizer.optimize()

if result is not None:
    best_parking_lot, search_radius = result
    # Create a visual map
    map_viz = optimizer.visualize_results(best_parking_lot, search_radius)

    # Save the map as an HTML file
    map_viz.save('parking_recommendation.html')

    # Print information about the best parking lot
    print(f"\nParking Lot Found in {search_radius}m: ")
    print(f"ID: {best_parking_lot['id']}")
    print(f"Location: ({best_parking_lot['lat']}, {best_parking_lot['lng']})")
    print(f"Hourly Rate: ${best_parking_lot['rate_hourly']}")
    print(f"Distance to destination: {best_parking_lot['distance']:.0f}m")
else:
    # If no parking lot is found, still create a map but only show the search area
    map_viz = optimizer.visualize_results(None, 1600)
    map_viz.save('parking_recommendation.html')



Parking Lot Found in 800m: 
ID: 16.0
Location: (43.651696, -79.383699)
Hourly Rate: $7.0
Distance to destination: 172m
